In [1]:
import sys
# !{sys.executable} -m pip install opencv-python
from IPython.core.debugger import set_trace
import cv2 
import numpy as np
from scipy import signal
from IPython.core.debugger import set_trace
from sklearn.metrics import mean_squared_error
from sklearn import preprocessing

In [2]:
def find_dft(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) 
    edged = cv2.Canny(gray, 30, 200) 
    contours, hierarchy = cv2.findContours(edged,  
        cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE) 
    if len(contours)!=1:
        return [],[],False
    contours1 = np.reshape(contours,[-1,2])
    contours1 = np.divide(contours1,np.repeat(np.reshape(np.add(np.power(contours1[:,0],2),np.power(contours1[:,1],2)),[-1,1]),2,axis = 1))

    u_hat = contours1[:,0]+1j*contours1[:,1]
    u_hat[~np.isnan(u_hat).any(axis=0)]
    template = np.fft.fft(u_hat)
#     template_unit = template/np.absolute(template)

    return template,contours,True

image = cv2.imread('./sample.png') 
targetImage = cv2.imread('./a3.png') 

sheight,swidth = image.shape[0],image.shape[1]
height,width = targetImage.shape[0],targetImage.shape[1]

cv2.waitKey(0) 
  
# Grayscale 
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) 
  
# Find Canny edges 
edged = cv2.Canny(gray, 30, 200) 
cv2.waitKey(0) 
  
# Finding Contours 
# Use a copy of the image e.g. edged.copy() 
# since findContours alters the image 
contours, hierarchy = cv2.findContours(edged,  
    cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE) 

template,_,_ = find_dft(image)
template = template[:30]
# template_abs = np.absolute(template)
# index = sorted(np.argsort(template_abs)[:50])
# template  = template[index]

# template_unit = np.divide(template,np.mean(np.absolute(template)))[:10]
temp_length = len(template)
# cv2.imshow('Canny Edges After Contouring', edged) 
# cv2.waitKey(0) 

print("Number of Contours found = " + str(len(contours))) 
  
# Draw all contours 
# -1 signifies drawing all contours 
cv2.drawContours(image, contours, -1, (0, 255, 0), 1) 
  
# cv2.imshow('Contours', image) 
# cv2.waitKey(0) 
cv2.destroyAllWindows() 



Number of Contours found = 1


In [3]:
# for h in range(sheight,height,5):
#     for w in range(swidth,width,5):
#         crop = targetImage[h-sheight:h,w-swidth:w]
#         stemplate,contour,checker = find_dft(crop)
# #         cv2.imshow('crops', crop) 
# #         cv2.waitKey(0) 
# #         cv2.destroyAllWindows() 
#         if checker and len(stemplate)>=temp_length and len(stemplate)>=80:
#             stemplate_abs = np.absolute(stemplate)
#             index1 = sorted(np.argsort(stemplate_abs)[:80])
#             stemplate_unit = np.divide(stemplate,np.absolute(stemplate))
#             stemplate_unit  = stemplate_unit[index1]
# #             dist = np.mean(np.power(np.absolute(template_unit - stemplate_unit),2))
#             dist = np.mean(np.power(np.imag(template_unit)-np.imag(stemplate_unit),2)+np.power(np.real(template_unit)-np.real(stemplate_unit),2))
# #             set_trace()
#             if dist:
#                 cv2.drawContours(crop, contour, -1, (0, 255, 0), 1) 
#                 cv2.imshow('crops', crop) 
#                 cv2.waitKey(0) 
#                 cv2.destroyAllWindows() 
#                 print(dist)
#         else:
#             continue
   

In [4]:
for h in range(sheight,height,5):
    for w in range(swidth,width,5):
        crop = targetImage[h-sheight:h,w-swidth:w]
        stemplate,contour,checker = find_dft(crop)
        
#         if checker and len(stemplate)>=temp_length and len(stemplate)>=80:
        if checker and len(stemplate)>=30:
            stemplate = stemplate[:30]
#             stemplate_unit = np.divide(stemplate,np.mean(np.absolute(stemplate)))[:10]
#             dist = np.absolute(signal.correlate(template,stemplate, mode='same') )
            dist1 = np.mean(np.power(np.imag(template)-np.imag(stemplate),2)+np.power(np.real(template)-np.real(stemplate),2))
            dist2 = np.mean(np.abs(np.absolute(template) - np.absolute(stemplate)))
            dist = dist2
#             cv2.drawContours(crop, contour[:40], -1, (0, 255, 0), 1) 
            dist = dist*100
            if dist<2 and not np.isnan(dist):
                cv2.imshow('crops', crop) 
                cv2.waitKey(0) 
                cv2.destroyAllWindows() 
                print(dist)


0.6398363821108363
1.2083961273165833
1.1113672354844708
0.8129698336248664


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in true_divide
  if __name__ == '__main__':


1.6405860521581062
1.8032357858256396
1.569898846820176
1.8547980172532068


In [5]:
np.mean(a)

NameError: name 'a' is not defined

In [ ]:
preprocessing.normalize([1,2,3], norm='l2',axis = 1)









In [ ]:
a= [1+1j, 7-5j]
np.absolute(a)